<a href="https://colab.research.google.com/github/hema-demo1/Hemavathi/blob/main/%22midterm_correction%22_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [14]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP, temperature, alertness, inspired_oxygen, performance_status):
    """
    Determine the severity of an individual's COVID-19 condition based on various vital signs and factors. The risk score is calculated using specific thresholds,
    and the corresponding risk is assigned based on the score.

    Args:
        sex (str): Gender assigned at birth.
        age (int): Age  in years.
        respiratory_rate (float): Respiratory rate in breaths per minute.
        oxygen_saturation (float): Oxygen saturation as a percent between 0 and 1.
        heart_rate (float): Heart rate in beats per minute.
        systolic_bp (float): Systolic BP in mmHg.
        temperature (float): Temperature in degrees C.
        alertness (str): Alertness as a string description.
        inspired_oxygen (str):Inspired Oxygen as as string description'.
        performance_status (str): Performance Status as a string description.

    >>> priest('male', 30, 16, 0.98, 75, 120, 36.8, 'alert', 'air', 'unrestricted normal activity')
    0.01
    >>> priest('female', 85, 28, 0.85, 135, 85, 39.2, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
    0.99
    >>> priest('male', 55, 22, 0.94, 95, 110, 37.2, 'alert', 'air', 'limited strenuous activity, can do light activity')
    0.34

    Returns:
    float: The risk score indicating the severity of COVID-19 for the individual.

    The score ranges are predefined and are utilized to assess the likelihood of adverse effects over the next 30 days.
    """

    sex = sex.lower()

    if sex not in ["male", "female"]:
        return None
    if not isinstance(age, int) or age < 0:
        return None
    if not isinstance(respiratory_rate, int) or respiratory_rate < 0:
        return None
    if not (isinstance(oxygen_saturation, float) and 0 <= oxygen_saturation <= 1):
        return None
    if not isinstance(heart_rate, int) or heart_rate < 0:
        return None
    if not isinstance(systolic_BP, int) or systolic_BP < 0:
        return None
    if not (isinstance(temperature, float) and temperature >= -100):
        return None

    alertness = alertness.lower()
    if alertness not in ["alert", "confused or not alert"]:
        return None

    inspired_oxygen = inspired_oxygen.lower()
    if inspired_oxygen not in ["air", "supplemental oxygen"]:
        return None

    performance_status = performance_status.lower()
    if performance_status not in [
        "unrestricted normal activity",
        "limited strenuous activity, can do light activity",
        "limited activity, can self-care",
        "bed/chair bound, no self-care"
    ]:
        return None

    sex_score = 1 if sex == "male" else 0

    if age >= 16 and age <= 49:
        age_score = 0
    elif age >= 50 and age <= 65:
        age_score = 2
    elif age >= 66 and age <= 80:
        age_score = 3
    elif age > 80:
        age_score = 4
    else:
        return None

    if respiratory_rate < 9:
        respiratory_rate_score = 3
    elif respiratory_rate >= 9 and respiratory_rate <= 11:
        respiratory_rate_score = 1
    elif respiratory_rate >= 12 and respiratory_rate <= 20:
        respiratory_rate_score = 0
    elif respiratory_rate >= 21 and respiratory_rate <= 24:
        respiratory_rate_score = 2
    else:
        respiratory_rate_score = 3

    if oxygen_saturation > 0.95:
        oxygen_saturation_score = 0
    elif oxygen_saturation >= 0.94 and oxygen_saturation <= 0.95:
        oxygen_saturation_score = 1
    elif oxygen_saturation >= 0.92 and oxygen_saturation <= 0.93:
        oxygen_saturation_score = 2
    else:
        oxygen_saturation_score = 3

    if heart_rate < 41:
        heart_rate_score = 3
    elif heart_rate >= 41 and heart_rate <= 50:
        heart_rate_score = 1
    elif heart_rate >= 51 and heart_rate <= 90:
        heart_rate_score = 0
    elif heart_rate >= 91 and heart_rate <= 110:
        heart_rate_score = 1
    elif heart_rate >= 111 and heart_rate <= 130:
        heart_rate_score = 2
    else:
        heart_rate_score = 3

    if systolic_BP < 91:
        systolic_BP_score = 3
    elif systolic_BP >= 91 and systolic_BP <= 100:
        systolic_BP_score = 2
    elif systolic_BP >= 101 and systolic_BP <= 110:
        systolic_BP_score = 1
    elif systolic_BP >= 111 and systolic_BP <= 219:
        systolic_BP_score = 0
    else:
        systolic_BP_score = 3

    if temperature < 35.1:
        temperature_score = 3
    elif temperature >= 35.1 and temperature <= 36.0:
        temperature_score = 1
    elif temperature >= 36.1 and temperature <= 38.0:
        temperature_score = 0
    elif temperature >= 38.1 and temperature <= 39.0:
        temperature_score = 1
    else:
        temperature_score = 2

    if alertness == "alert":
        alertness_score = 0
    elif alertness == "confused or not alert":
        alertness_score = 3
    else:
        return None

    if inspired_oxygen == "air":
        inspired_oxygen_score = 0
    elif inspired_oxygen == "supplemental oxygen":
        inspired_oxygen_score = 2
    else:
        return None

    if performance_status == "unrestricted normal activity":
        performance_status_score = 0
    elif performance_status == "limited strenuous activity, can do light activity":
        performance_status_score = 1
    elif performance_status == "limited activity, can self-care":
        performance_status_score = 2
    elif performance_status == "bed/chair bound, no self-care":
        performance_status_score = 3
    else:
        return None

    risk_score = sum([
        sex_score, age_score, respiratory_rate_score, oxygen_saturation_score,
        heart_rate_score, systolic_BP_score, temperature_score,
        alertness_score, inspired_oxygen_score, performance_status_score
    ])


    if risk_score >= 0 and risk_score <= 1:
        risk = 0.01
    elif risk_score >= 2 and risk_score <= 3:
        risk = 0.02
    elif risk_score == 4:
        risk = 0.03
    elif risk_score == 5:
        risk = 0.09
    elif risk_score == 6:
        risk = 0.15
    elif risk_score == 7:
        risk = 0.18
    elif risk_score == 8:
        risk = 0.22
    elif risk_score == 9:
        risk = 0.26
    elif risk_score == 10:
        risk = 0.29
    elif risk_score == 11:
        risk = 0.29
    elif risk_score == 12:
        risk = 0.38
    elif risk_score == 13:
        risk = 0.46
    elif risk_score == 14:
        risk = 0.47
    elif risk_score == 15:
        risk = 0.49
    elif risk_score == 16:
        risk = 0.55
    elif risk_score >= 17 and risk_score <= 25:
        risk = 0.59
    else:
        risk = 0.99

    return risk


result = priest('FEMALE', 40, 24, 0.96, 105, 115, 39.9, 'Alert', 'AIR', 'Unrestricted normal activity')
print(result)

0.09


In [15]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 30, 16, 0.98, 75, 120, 36.8, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.01
ok
Trying:
    priest('female', 85, 28, 0.85, 135, 85, 39.2, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
Expecting:
    0.99
**********************************************************************
File "__main__", line 20, in NoName
Failed example:
    priest('female', 85, 28, 0.85, 135, 85, 39.2, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
Expected:
    0.99
Got nothing
Trying:
    priest('male', 55, 22, 0.94, 95, 110, 37.2, 'alert', 'air', 'limited strenuous activity, can do light activity')
Expecting:
    0.34
**********************************************************************
File "__main__", line 22, in NoName
Failed example:
    priest('male', 55, 22, 0.94, 95, 110, 37.2, 'alert', 'air', 'limited strenuous activity, can do light activity')
Expected:
    0.34
Got:
    0.26


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [16]:
import requests


def find_hospital(age, sex, risk):
  """
  This function takes the risk score, age, and sex of the patient and figures out the nearest facility
   where the person can be taken in case of an emergency using the REST web services.

     >>> find_hospital(40, 'male', 0.18)
        'Southwest Hospital and Medical Center'
    >>> find_hospital(30, 'female', 0.09)
        'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(22, 'female', 0.92)
        'Emory Dunwoody Medical Center'
     """
  url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1'
  response = requests.get(url)
  hospital_name = response.json()['hospital']
  return hospital_name

In [17]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.18)
Expecting:
       'Southwest Hospital and Medical Center'
**********************************************************************
File "__main__", line 9, in NoName
Failed example:
    find_hospital(40, 'male', 0.18)
Expected:
       'Southwest Hospital and Medical Center'
Got:
    'Southwest Hospital and Medical Center'
Trying:
    find_hospital(30, 'female', 0.09)
Expecting:
        'Select Specialty Hospital - Northeast Atlanta'
**********************************************************************
File "__main__", line 11, in NoName
Failed example:
    find_hospital(30, 'female', 0.09)
Expected:
        'Select Specialty Hospital - Northeast Atlanta'
Got:
    'Southwest Hospital and Medical Center'
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
        'Emory Dunwoody Medical Center'
**********************************************************************
File "__main__", line 13, in NoName
Failed example:
 

## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [18]:
import requests

def get_address(hospital_name):
    """
    Fetches the hospital address from a JSON file based on the provided hospital name.

    Parameters:
    hospital_name (str): The name of the hospital.

    Returns:
    str or None: The address of the hospital. Returns None if the hospital name is not found.

    Examples:
    >>> fetch_hospital_address("Southwest Hospital and Medical Center")
        '501 FAIRBURN ROAD SW'
    >>> find_hospital(65, 'male', 0.18)
        'Wesley Woods Geriatric Hospital'
    >>> find_hospital(40, 'male', 0.18)
    'Southwest Hospital and Medical Center'

    """
    json_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    response = requests.get(json_url)

    if response.status_code == 200:
        data = response.json()

        if hospital_name in data:
            return data[hospital_name]["ADDRESS"]
    return None

In [19]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.18)
Expecting:
       'Southwest Hospital and Medical Center'
**********************************************************************
File "__main__", line 9, in NoName
Failed example:
    find_hospital(40, 'male', 0.18)
Expected:
       'Southwest Hospital and Medical Center'
Got:
    'Southwest Hospital and Medical Center'
Trying:
    find_hospital(30, 'female', 0.09)
Expecting:
        'Select Specialty Hospital - Northeast Atlanta'
**********************************************************************
File "__main__", line 11, in NoName
Failed example:
    find_hospital(30, 'female', 0.09)
Expected:
        'Select Specialty Hospital - Northeast Atlanta'
Got:
    'Southwest Hospital and Medical Center'
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
        'Emory Dunwoody Medical Center'
**********************************************************************
File "__main__", line 13, in NoName
Failed example:
 

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [20]:
import requests
import csv

def process_people(file_location):
    """
    (str) -> dict
    Processes patient data from the file in given location and retrieves risk, hospital and address information.
    """
    data = {}

    try:
        response = requests.get(file_location)
        response.raise_for_status()

        reader = csv.DictReader(response.text.splitlines(), delimiter='|')

        # Print the header to debug
        print("CSV Headers:", reader.fieldnames)

        for row in reader:
            if 'patient' not in row:
                print("Missing 'patient' column in row:", row)
                continue  # Skip this row if 'patient' key is not found
            patient_number = row['patient']
            sex = row['sex']
            age = int(row['age'])
            breath = float(row['breath'])
            o2sat = float(row['o2 sat'])
            heart = float(row['heart'])
            systolic = float(row['systolic bp'])
            temp = float(row['temp'])
            alertness = row['alertness']
            inspired = row['inspired']
            status = row['status']

            risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
            hospital = find_hospital(age, sex, risk)
            hospital_upper = hospital.upper()
            address = get_address(hospital_upper)

            patient_data = [
                sex,
                age,
                breath,
                o2sat,
                heart,
                systolic,
                temp,
                alertness,
                inspired,
                status,
                risk,
                hospital,
                address,
            ]

            data[patient_number] = patient_data

    except Exception as e:
        print("An error occurred while calling the existing functions:", e)

    return data

# Calling function "process_people"
file_location = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
processed_data = process_people(file_location)
print(processed_data)

CSV Headers: ['{']
Missing 'patient' column in row: {'{': '  "COOK MEDICAL CENTER":{'}
Missing 'patient' column in row: {'{': '    "City":"ADEL",'}
Missing 'patient' column in row: {'{': '    "TYPE":"GENERAL ACUTE CARE",'}
Missing 'patient' column in row: {'{': '    "ADDRESS":"706 NORTH PARRISH AVENUE",'}
Missing 'patient' column in row: {'{': '    "WEBSITE":"http:\\/\\/www.memorialofadel.com\\/body.cfm?id=364",'}
Missing 'patient' column in row: {'{': '    "BEDS":60,'}
Missing 'patient' column in row: {'{': '    "NAICS_DESC":"GENERAL MEDICAL AND SURGICAL HOSPITALS"'}
Missing 'patient' column in row: {'{': '  },'}
Missing 'patient' column in row: {'{': '  "PHOEBE PUTNEY MEMORIAL HOSPITAL":{'}
Missing 'patient' column in row: {'{': '    "City":"ALBANY",'}
Missing 'patient' column in row: {'{': '    "TYPE":"GENERAL ACUTE CARE",'}
Missing 'patient' column in row: {'{': '    "ADDRESS":"417 THIRD AVENUE",'}
Missing 'patient' column in row: {'{': '    "WEBSITE":"http:\\/\\/phoebeputney.com\\

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [21]:
import requests
import json

def retrieve_processed_data(file_url):
    response = requests.get(file_url)
    if response.status_code == 200:
        return json.loads(response.text)
    return None

processed_data_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
expected_data_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

processed_data = retrieve_processed_data(processed_data_url)
given_json_results = retrieve_processed_data(expected_data_url)

if processed_data and given_json_results:

    for patient_number, expected_data in given_json_results.items():
        if patient_number in processed_data:
            if processed_data[patient_number] == expected_data:
                print(f"Patient {patient_number}: Results match expected output.")
            else:
                print(f"Patient {patient_number}: Results do not match expected output.")
        else:
            print(f"Patient {patient_number} not found in the processed data.")

    for patient_number in processed_data:
        if patient_number not in given_json_results:
            print(f"Patient {patient_number} in processed data but not in expected results.")

    for patient_number in given_json_results:
        if patient_number not in processed_data:
            print(f"Patient {patient_number} in expected results but not in processed data.")

else:
    print("Failed to retrieve processed or expected data.")


Patient E9559: Results match expected output.
Patient E9385: Results match expected output.
Patient E3067: Results match expected output.
Patient E9422: Results match expected output.
Patient E8661: Results match expected output.
Patient E6235: Results match expected output.
Patient E4451: Results match expected output.
Patient E8433: Results match expected output.
Patient E7593: Results match expected output.
Patient E3296: Results match expected output.
Patient E4157: Results match expected output.
Patient E7702: Results match expected output.
Patient E8158: Results match expected output.
Patient E4795: Results match expected output.
Patient E8800: Results match expected output.
Patient E4855: Results match expected output.
Patient E9691: Results match expected output.
Patient E4535: Results match expected output.
Patient E2360: Results match expected output.
Patient E3447: Results match expected output.
Patient E8208: Results match expected output.
Patient E4428: Results match expec

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---